In [1]:
import circle_cal.model as ccm
from workalendar.usa import UnitedStates, Indiana
from workalendar.core import ChristianMixin
from datetime import datetime, date, time
from datetime import timedelta
import calendar

In [2]:
y24 = ccm.Year(2024)

In [3]:
y24.THETA_PER_DAY

0.9836065573770492

In [8]:
ccm._set_month(datetime(2024, 1, 1), 12)

datetime.datetime(2025, 1, 1, 0, 0)

In [5]:
for m in y24:
    for d in m:
        print(d)

(2024-01-01 00:00:00, 2024-01-31 00:00:00)
(2024-02-01 00:00:00, 2024-02-29 00:00:00)
(2024-02-01 00:00:00, 2024-02-29 00:00:00)
(2024-03-01 00:00:00, 2024-03-31 00:00:00)
(2024-03-01 00:00:00, 2024-03-31 00:00:00)
(2024-04-01 00:00:00, 2024-04-30 00:00:00)
(2024-04-01 00:00:00, 2024-04-30 00:00:00)
(2024-05-01 00:00:00, 2024-05-31 00:00:00)
(2024-05-01 00:00:00, 2024-05-31 00:00:00)
(2024-06-01 00:00:00, 2024-06-30 00:00:00)
(2024-06-01 00:00:00, 2024-06-30 00:00:00)
(2024-07-01 00:00:00, 2024-07-31 00:00:00)
(2024-07-01 00:00:00, 2024-07-31 00:00:00)
(2024-08-01 00:00:00, 2024-08-31 00:00:00)
(2024-08-01 00:00:00, 2024-08-31 00:00:00)
(2024-09-01 00:00:00, 2024-09-30 00:00:00)
(2024-09-01 00:00:00, 2024-09-30 00:00:00)
(2024-10-01 00:00:00, 2024-10-31 00:00:00)
(2024-10-01 00:00:00, 2024-10-31 00:00:00)
(2024-11-01 00:00:00, 2024-11-30 00:00:00)


In [8]:
m.part_unit()

'day'

In [9]:
for d in m:
    print(d)

(2024-12-01 00:00:00, 2024-12-31 00:00:00)


In [8]:
d + timedelta(days=31)

datetime.date(2024, 2, 1)

In [9]:
y24.len_by_days()

366.0

In [13]:
calendar.month_name[1]

'January'

In [7]:
dt = datetime(2024, 1, 1)

In [8]:
dt.year = 2025

AttributeError: attribute 'year' of 'datetime.date' objects is not writable

In [13]:
dict(**dt)

TypeError: dict() argument after ** must be a mapping, not datetime.datetime

In [1]:
d = {"a": 1}

In [6]:
d.update({"b": 2})

In [7]:
d

{'a': 1, 'b': 2}

In [8]:
13 % 12

1

In [9]:
10 % 12

10

In [12]:
calendar.month_name[1]

'January'

In [2]:
11 % 12

11

In [4]:
12 % 12

0

In [ ]:
12 